In [1]:
import pandas as pd
import numpy as np
# Create feature view for retrieval model (training data)
# Generate UUID as a key, in conjunction to ufl.edu email
# 
#Seperate Mentor from Mentee 

# The candidate is a mentor if 'prefered_littles" is empty, and 'prefered_bigs' has a value
# vice versa for mentee is the reverse 



In [ ]:
main_df = pd.read_csv('./data/embedded_data.csv')
print(main_df['preferred_littles'],  main_df['preferred_bigs'])

In [ ]:
# Seperating Mentors from Mentees 
main_df['preferred_littles'] = pd.to_numeric(main_df['preferred_littles'], errors='coerce')
main_df['preferred_bigs'] = pd.to_numeric(main_df['preferred_bigs'], errors='coerce')


In [ ]:
# check mentors vs. mentees
is_mentor_condition = main_df['preferred_littles'].notna() & main_df['preferred_bigs'].isna()
is_mentee_condition = main_df['preferred_bigs'].notna() & main_df['preferred_littles'].isna()

mentors_df = main_df[is_mentor_condition].copy()
mentees_df = main_df[is_mentee_condition].copy()

print(f"Identified {len(mentors_df)} mentors.")
print(f"Identified {len(mentees_df)} mentees.")



In [ ]:
 # Generate UUID as a key, in conjunction to ufl.edu email
import uuid

# Generate unqiue user_id to the data frame before splitting
# we use hex so it is just unicode characters
main_df['user_id'] = [uuid.uuid4().hex for _ in range(len(main_df))]

# Display the first few rows with the new ID
print("--- Mentors DataFrame with user_id ---")
print(mentors_df[['user_id', 'ufl_email']].head())

print("\n--- Mentees DataFrame with user_id ---")
print(mentees_df[['user_id', 'ufl_email']].head())

In [ ]:
# Create Paired Feature view for retrieval
## Initialize a map for pairs data <mentee, mentor>
mentees_df['key'] = 1
mentors_df['key'] = 1

# Perform cross join or Cartesin Product (x)
retrieval_df = pd.merge(
    mentees_df,
    mentors_df,
    on='key',
    suffixes=('_mentee', '_mentor')).drop('key', axis=1)

# This creates a clear distinction between mentee and mentor features
mentee_cols = {col: f"mentee_{col}" for col in mentees_df.columns}
mentor_cols = {col: f"mentor_{col}" for col in mentors_df.columns}

mentee_cols

In [ ]:
# Unpacking for feature view 
# e.g. year_mentee, major_mentor

retrieval_df = retrieval_df.rename(columns={**mentee_cols, **mentor_cols})

retrieval_df.head()

In [ ]:

# convert DF to a dictionary for tensors
retrieval_dict = {name: np.array(value) for name, value in retrieval_df.items()}
